In [1]:
# Read in some data from the RDSF

In [ ]:
# Crop out the jaw

In [3]:
# Segment out the jaw

In [4]:
# Show a mesh of the jaw

In [5]:
# Show the greyscale values of the jaw pixels

In [ ]:
# Show a histogram of the greyscale values in the jaw